##### *Python libraries used in this notebook*

In [1]:
%matplotlib inline
import control.matlab as control
from ipywidgets import interact, fixed
import matplotlib.pyplot as plt
import numpy as np

# PID controller

## Tuning of PID controller using Ziegler-Nichols method

### *Abstract*
This notebook demonstrates а tuning of a PID controller using Ziegler-Nichols method.

### *Table of contents*
1. Description of a control systems
2. Description of a control loop
    1. Open loop control
    2. Closed loop control
3. Introduction of PID controller
    1. PID controller theory
        1. Proportional component
        2. Integral component
        3. Derivative component 
        4. Laplace domain transfer functions 
    2. Tuning methods


### *1. Description of a control systems*
A **control system** is a system that provides the desired response by controlling the output. Explained in another way, this is a set of mechanical and electronic devices which regulate, manage, and command other devices or systems using control loops. The following figure shows the simple block diagram of a control system.

<img src="imgs\ControlSystemBlockDiagram.jpg" width=500 hight=500 />

Controls are classified with respect to:
> - technique involved to perform control (i.e. human/machines): manual/automatic control
> - creation of control system (i.e. natural / manmade): natural/ manmade /combinational
control
> - Time dependence of output variable (i.e. constant/changing): regulator/servo, (also known as
regulating/tracking control)
> - Time-type control (Continuous/ discrete): Continuous/Discrete time control
> - Linearity of the control system (linear/nonlinear): Linear/nonlinear control system
> - Based on the number of input output (SISO, MISO) system
> - fundamental structure / configuration of the control (i.e. the information used for computing the
control): open loop/feedback control, (also known as open loop/closed loop control)
    
This definition makes sense if you know what a **control loop** is, so let's define what is a control loop in the next cell.

### 2. *Description of a control loop*
A **control loop** is the configuration of the control system. This configuration consists of all the physical components and control functions needed to adjust or maintain the value of a measured process variable $(PV)$ to your desired value, or setpoint $(SP)$. To do this, your loop needs at least three basic elements – an input, a controller, and an output. The following figures show some process control loops in modern industrial automation.

<img src="imgs\ControlLoopExample.jpg" width=1000 hight=800 />

There are two types of control loops: 
> - Open loop control
> - Closed loop control

Let's look at the two types in the next cells.

### *2.A. Open loop control*
An open loop control is applied to achieve the desired system response using a controller or an actuator without feedback. This means that the оpen loop systems do not monitor or correct the output for disturbances. The desired process response is applied to a controller and it produces an actuating signal or controlling signal (output). This signal is given as an input to a plant or process which is to be controlled. So, the plant produces an output, which is controlled. The following figure shows the simple block diagram of a control system.

<img src="imgs\OpenLoop.jpg" width=600 hight=600 />

Advantages of open Loop control:
> - These systems are simple in construction and design.
> - These systems are economical.
> - These systems are easy from a maintenance point of view.
> - Usually, these systems are not much troubled with problems of stability.
> - These systems are convenient to use when output is difficult to measure.

Disadvantages of open loop control:
> - These systems are not accurate and reliable because of their accuracy depends on the accuracy of
the calibration.
> - In these systems, inaccuracy results are obtained with parameter variations i.e. internal
disturbances.
> - Recalibration of the controller is required from time to time for maintaining quality and accuracy.

### *2.B. Closed loop control*
A closed loop control is used to achieve the desired system response using a controller with the output measurement as a feedback signal. The use of feedback enables us to improve system performance at the cost of introducing the measurement noise and stability problem. This means that the closed loop systems monitor the output and compare it to the input. If an error is detected, the system corrects the output and hence corrects the effects of disturbances. The following figure shows the simple block diagram of a control system.

<img src="imgs\ClosedLoop.jpg" width=500 hight=500 />

Advantages of closed loop control:
> - In these systems, accuracy is very high due to the correction of any arising error.
> - Since these systems sense environmental changes as well as internal disturbances, the errors are
modified.
> - There is a reduced effect of nonlinearity in these systems.
> - Systems have high bandwidth i.e. high operating frequency zone.
> - There are facilities for automation in these systems.

Disadvantages of closed loop control:
> - Systems are complicated in design and, hence, costlier.
> - Systems may be unstable.

After we already know what a control system is and what types of configurations it has, maybe it is time to understand what mechanisms (methods) we use in modern process industrial automation. Let's look at what a PID controller is in the following cells.

### *3. Introduction of PID controller*
A **Proportional-Integral-Derivative controller (PID controller)** is a control loop feedback mechanism widely used in industrial control systems. A PID controller calculates an error value as the difference between a measured process variable and a desired setpoint. The controller attempts to minimize the error by adjusting the process through use of a manipulated variable. The following photos show PID controllers used in the industry

<img src="imgs\PIDControllers.jpg" width=500 hight=500 />

The PID controller algorithm involves three separate constant parameters, and is accordingly sometimes called **three-term control**: the proportional, the integral and derivative values, denoted **P**, **I**, and **D**. Simply put, these values can be interpreted in terms of time: **P** depends on the ***present*** error, **I** on the accumulation of ***past*** errors, and **D** is a prediction of ***future*** errors, based on current rate of change. The weighted sum of these three actions is used to adjust the process via a control element such as the position of a control valve, a damper, or the power supplied to a heating element.

In the absence of knowledge of the underlying process, a PID controller has historically been considered to be the most useful controller. By tuning the three parameters in the PID controller algorithm, the controller can provide control action designed for specific process requirements. The response of the controller can be described in terms of the responsiveness of the controller to an error, the degree to which the controller overshoots the setpoint, and the degree of system oscillation. Note that the use of the PID algorithm for control does not guarantee optimal control of the system or system stability.

Some applications may require using only one or two actions to provide the appropriate system control. This is achieved by setting the other parameters to zero. A PID controller will be called a **P**, **PI** or **PD** controller in the absence of the respective control actions. PI controllers are fairly common, since derivative action is sensitive to measurement noise, whereas the absence of an integral term may prevent the system from reaching its target value due to the control action.

Let's see  the fundamentals behind the design of every PID controller.

### *3.A. PID controller theory*
PID **control theory** is a branch of mathematics and engineering with the purpose of modifying the behavior of a dynamic system with inputs (e.g. input from a temperature sensor) that changes over time. The main objective of control theory is stability. Usually, (in closed loop systems) stability is achieved by continually taking measurements and making adjustments to the system.

The PID control scheme is named after its three correcting terms, whose sum constitutes the manipulated variable $(MV)$. The proportional, integral, and derivative terms are summed to calculate the output of the PID controller. Defining $\large{u(t)}$ as the controller output, Controller manufacturers arrange the Proportional, Integral, and Derivative modes into three different controller algorithms or controller structures. These are called Interactive, Noninteractive, and Parallel algorithms. Some controller manufacturers allow you to choose between different controller algorithms as a configuration option in the controller software. The PID Algorithms are:

***Interactive algorithm:***

The oldest controller algorithm is called the Series, Classical, Real or Interactive algorithm.  The original pneumatic and electronic controllers had this algorithm and it is still found it in many controllers today.  The Ziegler-Nichols PID tuning rules were developed for this controller algorithm.

<img src="imgs\series.png" width=600 hight=600 />

$$\large{u(t) = K_c \left[ e(t) + \frac{1}{T_i}\int_0^t e(\tau)\ d\tau\right] + \left[1 + T_d\frac{d}{dt} e(t)\right]\tag{1}}$$

***Noninteractive algorithm:***

The Noninteractive algorithm is also called the Ideal, Standard or ISA algorithm. The Cohen-Coon and Lambda PID tuning rules were designed for this algorithm. If no derivative is used (i.e. Td = 0), the interactive and noninteractive controller algorithms are identical.

<img src="imgs\ideal.png" width=480 hight=480 />

$$\large{u(t) = K_c \left[ e(t) + \frac{1}{T_i}\int_0^t e(\tau)\ d\tau + T_d\frac{d}{dt} e(t)\right]\tag{2}}$$

***Parallel algorithm:***

Some academic textbooks discuss the parallel form of PID controller, but it is also used in some DCSs and PLCs. This algorithm is simple to understand, but not intuitive to tune. The reason is that it has no controller gain (affecting all three control modes), it has a proportional gain instead (affecting only the proportional mode). Adjusting the proportional gain should be supplemented by adjusting the integral and derivative settings at the same time. Try to not use this controller algorithm if possible (in some DCSs it is an option, so select the alternative).

<img src="imgs\parallel.png" width=450 hight=450 />

$$\large{u(t) = K_p e(t) + K_i\int_0^t e(\tau)\ d\tau + K_d\frac{d}{dt} e(t)\tag{3}}$$

***Where***:

$u(t)$ - Controller output

$T_i$ - Integral time

$T_d$ - Derivative time

$K_c$ - Controller gain

$K_p = K_c$ - Propotional gain

$K_i = \frac{K_c}{T_i}$ - Integral gain

$K_d = K_c T_d$ - Derivative gain

$e(t) = y_{sp}(t) - y(t)$ - Control error, which is the difference between the desired $(SP)$ process state, $y_{sp}(t)$, and the measured $(PV)$ process state, $y(t)$

Let's also look at the individual components of the PID controller.

### *3.A.a. Proportional component*
The proportional component produces an output value that is proportional to the current error value. The proportional response can be adjusted by multiplying the error by a constant $K_p$, called the proportional gain constant. The proportional term is given by:

$$\large{P_{out} = K_p e(t)}$$

A high proportional gain results in a large change in the output for a given change in the error. If the proportional gain is too high, the system can become unstable. In contrast, a small gain results in a small output response to a large input error, and a less responsive or less sensitive controller. If the proportional gain is too low, the control action may be too small when responding to system disturbances. Tuning theory and industrial practice indicate that the proportional term should contribute to the bulk of the output change.

### *3.A.b. Integral component*

### *3.A.c. Derivative component*

### *3.A.d. Laplace domain transfer functions*

### *3.B. Tuning methods*